# Function for Decision Tree

In [ ]:
import numpy as np
import pandas as pd
class Node(object):
   def __init__(self):
       self.value =None
       self.left =None
       self.right =None
       self.is_leaf=False
class decision_tree(object):
    def __init__(self):
        self.tree_=None
    def clac_max_infogain(self, X, y):

        features = X.columns
        best_feature = None
        best_infogain = [float('-inf')]
        entP = self.calc_entropy(y)
        for feature_name in features:
            info_gain = self.calc_infogain(X[feature_name], y, entP)
            if info_gain[0] > best_infogain[0]:
                best_feature = feature_name
                best_infogain = info_gain

        return best_feature, best_infogain

    # def info_gain(self, feature, y, entP):

    #     m = y.shape[0]
    #     uni_value = pd.unique(feature)

    #     ent = 0
    #     for value in uni_value:
    #         Dv = y[feature == value]  #
    #         # 这里又使用了broadcast
    #         ent(Dv)
    #         ent += Dv.shape[0] / m * ent_sub

    #     gain = entP - ent  # 原书中4.2式
    #     return [gain]

    def calc_infogain(self, feature, y, entP):
        total = y.shape[0]
        uni_value = pd.unique(feature)
        ent = 0
        info_gain=0
        for value in uni_value:
            y_feature =y[feature==value]
            # the size of p= [1,2], elegent
            ent_split = self.calc_entropy(y_feature)
            # 集成一下
            ent+= y_feature.shape[0]/total*ent_split
        info_gain=entP-ent
        return [info_gain]

    def calc_entropy(self, y):
        # 这是用了broad cast
        # p 1    0.533333
        # 0    0.466667
        p = pd.value_counts(y) / y.shape[0]  # 计算各类样本所占比率
        # print("p",p)
        ent = np.sum(-p * np.log2(p))
        return ent
        

    # def fit(self, X_train, y_train):


    #     self.tree_ = self.generate_tree(X_train, y_train)


    #     return self

    def generate_tree(self, X, y):
        # declare a class
        tree=Node()
        # use pandas, means y is all unique
        # print("y",y)

        if y.nunique()==1:
            tree.value= y.values[0]
            tree.is_leaf=True
            return tree
        # if y.empty:
        #     print("empty")
        #     return y
        # clac info_gain
        feature_criteria, info_gain = self.clac_max_infogain(X, y)
        feature_values = X.loc[:, feature_criteria]
        tree.value=feature_criteria
        sub_x = X.drop(feature_criteria, axis=1)
        # here assume feature has only 2 ways
        # yes
        # need to review the ooj of python
        tree.left = self.generate_tree(sub_x[feature_values==1], y[feature_values==1])
        tree.right = self.generate_tree(sub_x[feature_values==0], y[feature_values==0])
        return tree
    def predict(self, X,tree):
        for i in range(X.shape[0]):
            ret = self.predict_single(X.iloc[i, :],tree)
            print(ret)
            # ret_list=[]
            # ret_list.append[ret]
        # return ret_list
    def predict_single(self, x, tree):
        if(tree==None):
            print("train a tree firstly")
            # print(tree.value)
        # print(x[tree.value])
        # print("x[2]",x[2])
        # feature_value = x.loc[:, tree.value]
        # print(feature_value[0]==1)
        if(tree.is_leaf==True):
            # print(tree.value)
            return tree.value
        elif(x[tree.value]==1):
            ret = self.predict_single(x, tree.left)
            return ret
        elif(x[tree.value]==0):
            ret= self.predict_single(x, tree.right)
            return ret
        

# build.fit(data3.iloc[:, :5], data3.iloc[:, 5])


# print(data3['smoke'])
#####Training the tree
# tree = DecisionTree(criterion='infogain')


##test for predict_single
# test = pd.DataFrame(np.zeros((1,5), dtype=int),columns=["chest pain" , "male" ,"smoke", "drink", "exercise"])
# print(test)
# columns = list(test.columns)
# print(columns.index("smoke"))
# feature_values = test.loc[:, "smoke"]
# print(feature_values==1)
# predict= build.predict_single(test,tree)

# file_name=  '/home/aistudio/work/data/hw2-decision-tree-input_testing.txt'
# pd_file_name =file_name.replace('.txt','.csv')
# shutil.copy(file_name, pd_file_name)

# test_data = pd.read_csv(pd_file_name)
# print(test_data)

# predict= build.predict(test,tree)
# print(predict)




# Training

In [52]:
import shutil
####Read_data
file_name=  '/home/aistudio/work/data/hw2-decision-tree-input.txt'
pd_file_name =file_name.replace('.txt','.csv')
shutil.copy(file_name, pd_file_name)

data3 = pd.read_csv(pd_file_name)
print(data3)
build = decision_tree()
tree = build.generate_tree(data3.iloc[:, :5], data3.iloc[:, 5])
# print(tree.value)
# print(tree.left.value)
# print(tree.right.value)
# print(tree.left.left.value)
# print(tree.left.right.value)

    chest pain  male  smoke  drink  exercise  heart attack
0            1     1      1      1         0             1
1            1     0      1      1         0             1
2            1     0      0      0         1             0
3            1     1      1      1         1             1
4            1     0      1      0         0             1
5            1     1      1      0         0             1
6            1     1      1      0         1             1
7            0     1      1      1         0             1
8            0     0      1      1         0             1
9            0     1      1      0         0             0
10           0     1      0      1         0             0
11           0     1      1      0         1             0
12           0     0      1      0         1             0
13           0     0      0      1         1             0
14           0     1      0      1         1             0


# Output the tree

![](https://ai-studio-static-online.cdn.bcebos.com/016d8d0b211d4e118233297757ece426166f2b5c46e54af2935ecf4afa27b994)



In [54]:
# import treePlottter

#####/Print & Plot the tree
result={0:"no heart attack",1:"heart attack"}
label_dic={0:"no ",1:"yes"}


formatter= \
"The root node is attribute ___%r. Its left edge has label ___%r.  \
Its right edge has label ___%r. Its left child node’s attribute is ____%r. \
Its right child node’s attribute is _____%r."
# print(tree.value)
# print(tree.left.value)
# print(tree.right.value)
# print(tree.left.left.value)
# print(tree.left.right.value)

print(formatter  \
    %(tree.value ,label_dic[1], label_dic[0], \
    [result[tree.left.value] if tree.left.is_leaf else tree.left.value], \
    [result[tree.right.value] if tree.right.is_leaf else tree.left.value]
    ))


The root node is attribute ___'smoke'. Its left edge has label ___'yes'.  Its right edge has label ___'no '. Its left child node’s attribute is ____['chest pain']. Its right child node’s attribute is _____['no heart attack'].


# Testing

In [55]:
#####predict
####Read_data
file_name=  '/home/aistudio/work/data/hw2-decision-tree-input_testing.txt'
pd_file_name =file_name.replace('.txt','.csv')
shutil.copy(file_name, pd_file_name)

test_data = pd.read_csv(pd_file_name)
print(test_data)

print("Predict result:", build.predict(test_data, tree))

    chest pain  male  smoke  drink  exercise  heart attack
0            1     1      1      1         0             1
1            0     0      1      1         0             1
2            1     1      0      0         1             0
3            1     1      0      1         1             1
4            1     0      1      1         0             1
5            1     1      1      0         1             1
6            1     1      1      0         1             1
7            0     1      1      0         0             1
8            0     0      0      1         0             1
9            0     0      1      0         0             0
10           1     1      0      1         0             0
11           0     0      1      0         1             0
12           0     0      0      0         1             0
13           0     0      0      0         1             0
14           0     1      0      1         0             0
1
1
0
0
1
1
1
0
0
0
0
0
0
0
0
Predict result: None
